In [1]:
import piq
import datasets
import torch
from torchvision import transforms
from datasets import Dataset
from datasets import load_dataset
from transformers import pipeline
from evaluate import evaluator

In [2]:
def pil_to_pt(img):
    t = transforms.functional.pil_to_tensor(img)
    t = t.to(torch.float)
    t = t/255
    t = t.unsqueeze(0)
    return t

In [3]:
vit_b = pipeline("image-classification", model="google/vit-base-patch16-224")
task_evaluator = evaluator("image-classification")
clip_iqa = piq.CLIPIQA()
dataset = load_dataset("imagenet-1k")

In [ ]:
for i_sample in range(dataset['validation'].num_rows):
    sample = Dataset.from_dict(dataset['validation'][i_sample:i_sample+1])
    img = sample[0]['image']
    x = pil_to_pt(img)
    quality = clip_iqa(x)
    eval_results = task_evaluator.compute(
        model_or_pipeline=vit_b,
        data=sample,
        metric="accuracy",
        label_mapping=vit_b.model.config.label2id,
    )
    correct = eval_results['accuracy'] > 0.5
    print(quality)
    print(correct)
    if i_sample == 10:
        break

tensor([[0.9852]])
True
tensor([[0.6878]])
False
